In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [2]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We will be working with the same medical data used in the `pw` miniproject, but will be leveraging the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [3]:
!mkdir dw-data
!aws s3 sync s3://dataincubator-wqu/dwdata-ease/ ./dw-data

mkdir: cannot create directory ‘dw-data’: File exists


In [4]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [5]:
import pandas as pd
import numpy as np
import gzip

In [6]:
#load 2017 data
scripts_file = gzip.open('./dw-data/201701scripts_sample.csv.gz', 'r')
    
scripts = pd.read_csv(scripts_file)
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [7]:
scripts_file_16 = gzip.open('./dw-data/201606scripts_sample.csv.gz', 'r')
    
scripts_16 = pd.read_csv(scripts_file_16)
scripts_16.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85638,0301011R0,Salamol_Inha 100mcg (200 D) CFF (Teva),2,2.92,2.73,2
1,N85638,0301011R0,Easyhaler_Salbutamol Sulf 200mcg (200D),1,6.63,6.15,1
2,N85638,0301020I0,Ipratrop Brom_Inh Soln 500mcg/2ml Ud,1,1.77,1.75,12
3,N85638,0301020I0,Ipratrop Brom_Inh Soln 250mcg/1ml Ud,1,4.47,4.15,20
4,N85638,0302000C0,Clenil Modulite_Inha 50mcg (200D),1,3.70,3.44,1


In [8]:
practices_file = gzip.open('./dw-data/practices.csv.gz', 'r')
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = pd.read_csv(practices_file, names = col_names)

# practices = practices[['code', 'name']].sort_values (by = ['name'], ascending = True) # insert
# practices = practices [~practices.duplicated(['code'])] # insert

practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [9]:
chem = pd.read_csv(gzip.open('./dw-data/chem.csv.gz', 'r'))
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [10]:
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [11]:
scripts_sum = scripts[['items', 'nic', 'act_cost', 'quantity']].agg(['sum'])
scripts_sum

,items,nic,act_cost,quantity
sum,8888304,71100424.84,66164096.12,721457006


In [12]:
scripts_des = scripts.describe()
scripts_des

,items,nic,act_cost,quantity
count,973193.000000,973193.000000,973193.000000,973193.000000
mean,9.133136,73.058915,67.986613,741.329835
std,29.204198,188.070257,174.401703,3665.426958
min,1.000000,0.000000,0.040000,0.000000
25%,1.000000,7.800000,7.330000,28.000000
50%,2.000000,22.640000,21.220000,100.000000
75%,6.000000,65.000000,60.670000,350.000000
max,2384.000000,16320.000000,15108.320000,577720.000000


In [13]:
scripts_output = pd.concat([scripts_sum, scripts_des.drop(['count','min','max'])])
scripts_output

,items,nic,act_cost,quantity
sum,8.888304e+06,7.110042e+07,6.616410e+07,7.214570e+08
mean,9.133136e+00,7.305892e+01,6.798661e+01,7.413298e+02
std,2.920420e+01,1.880703e+02,1.744017e+02,3.665427e+03
25%,1.000000e+00,7.800000e+00,7.330000e+00,2.800000e+01
50%,2.000000e+00,2.264000e+01,2.122000e+01,1.000000e+02
75%,6.000000e+00,6.500000e+01,6.067000e+01,3.500000e+02


In [14]:
act_cost_l = []
for i in scripts_output['act_cost']:
    act_cost_l.append(i)
    
nic_l = []
for i in scripts_output['nic']:
    nic_l.append(i)

items_l = []
for i in scripts_output['items']:
    items_l.append(i)
    
quantity_l = []
for i in scripts_output['quantity']:
    quantity_l.append(i)

#print(output_list_of_tuples)   
output_list = [('items', tuple(items_l)), ('quantity', tuple(quantity_l)), ('nic', tuple(nic_l)),('act_cost', tuple(act_cost_l))]

In [15]:
def summary_stats():
    return output_list
    #return [('items', (1,) * 6), ('quantity', (1,) * 6), ('nic', (1,) * 6), ('act_cost', (1,) * 6)]

In [16]:
grader.score.dw__summary_statistics(summary_stats)

Your score:  1.0


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [17]:
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [18]:
scripts_gp = scripts.groupby('bnf_name')['items'].count()
scripts_gp

bnf_name
365 Film 10cm x 12cm VP Adh Film Dress        1
365 Non Adherent 10cm x 10cm Pfa Plas Fa      3
365 Non Adherent 10cm x 20cm Pfa Plas Fa      1
365 Non Woven Island 8cm x 10cm Adh Dres      1
365 Transpt Island 5cm x 7.2cm VP Adh Fi      2
365 Transpt Island 8.5cm x 15.5cm VP Adh      1
3M Micropore Silicone 2.5cm x 5m Surg Ad     10
3M Micropore Silicone 5cm x 5m Surg Adh       7
3m Health Care_Cavilon Durable Barrier C    816
3m Health Care_Cavilon No Sting 1ml Barr    223
3m Health Care_Cavilon No Sting 3ml Barr     94
3m Health Care_Cavilon No Sting Barrier     412
4Head_Top Headache Relief Stick               4
A & P_Infants Pdrs                            5
A.S Saliva Orthana Spy 50ml (App)           121
A.S Saliva Orthana Spy Refill 500ml (App      6
A.T. 10_Soln 0.25mg/1ml                       1
A2A Spacer                                   23
A2A Spacer + Sml/Med Mask                    62
AAA_Sore Throat A/Spy 7.5g                    4
ACC-600_Tab Eff                

In [19]:
sums = scripts.groupby('bnf_name').sum()['items']
sums.head()

bnf_name
365 Film 10cm x 12cm VP Adh Film Dress      2
365 Non Adherent 10cm x 10cm Pfa Plas Fa    3
365 Non Adherent 10cm x 20cm Pfa Plas Fa    1
365 Non Woven Island 8cm x 10cm Adh Dres    1
365 Transpt Island 5cm x 7.2cm VP Adh Fi    2
Name: items, dtype: int64

In [20]:
def most_common_item():
    
    sums = scripts.groupby('bnf_name').sum()['items']
    item = (sums.idxmax(), sums.max())
    return [item]

In [21]:
answer = most_common_item()
answer

[('Omeprazole_Cap E/C 20mg', 218583)]

In [22]:
#items = scripts.groupby(['bnf_name'], sort=False)['items'].sum()
#items_df = pd.DataFrame({'bnf_name':items.index, 'items':items.values})
#a = items_df.loc[items_df['items'].idxmax()]
#most_common_item=[]
#emptytuple=()
#emptytuple=a[0],a[1]
#most_common_item.append(emptytuple)
#print(most_common_item)

In [23]:
#most_common_item = [("", 0)]

In [24]:
grader.score.dw__most_common_item(most_common_item)

Error!
You have been rate limited for exceeding the limit of 1 per 1 second.
Please slow down your submission rate.


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [25]:
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [26]:
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [27]:
srt = practices.sort_values('post_code').drop_duplicates('code', keep='first')
srt.head()

,code,name,addr_1,addr_2,borough,village,post_code
1896,E82060,PARKBURY HOUSE SURGERY,PARKBURY HOUSE,ST.PETERS STREET,ST.ALBANS,HERTFORDSHIRE,AL1 3HD
1871,E82031,MALTINGS SURGERY,THE MALTINGS SURGERY,8-14 VICTORIA STREET,ST ALBANS,HERTFORDSHIRE,AL1 3JB
1849,E82004,HATFIELD ROAD SURGERY,61 HATFIELD ROAD,NaN,ST.ALBANS,HERTFORDSHIRE,AL1 4JE
1848,E82002,WRAFTON HOUSE SURGERY,WRAFTON HOUSE SURGERY,9/11 WELLFIELD ROAD,HATFIELD,HERTFORDSHIRE,AL10 0BS
9812,Y05146,HCT LYMPHOEDEMA AT WEST ESSEX CCG,QUEENSWAY HEALTH CENTRE,QUEENSWAY,HATFIELD,HERTFORDSHIRE,AL10 0LF


In [28]:
merged = scripts.merge(srt, left_on='practice', right_on='code')
merged.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,code,name,addr_1,addr_2,borough,village,post_code
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF


In [29]:
total_items_by_postcode = merged.groupby(['post_code','bnf_name'])[['items']].sum()
total_items_by_postcode.head()

items
post_code bnf_name                                       
B11 4BW   3m Health Care_Cavilon Durable Barrier C      7
          3m Health Care_Cavilon No Sting Barrier       2
          Abasaglar KwikPen_100u/ml 3ml Pf Pen          2
          Abidec_Dps                                   63
          Able Spacer + Sml/Med Mask                    1

In [30]:
total_items_by_postcode.reset_index(inplace=True)
total_items_by_postcode.head()

,post_code,bnf_name,items
0,B11 4BW,3m Health Care_Cavilon Durable Barrier C,7
1,B11 4BW,3m Health Care_Cavilon No Sting Barrier,2
2,B11 4BW,Abasaglar KwikPen_100u/ml 3ml Pf Pen,2
3,B11 4BW,Abidec_Dps,63
4,B11 4BW,Able Spacer + Sml/Med Mask,1


In [31]:
total_items_max = total_items_by_postcode.groupby('post_code')[['items']].max()
total_items_max.head()

,items
post_code,
B11 4BW,706
B12 9LP,425
B18 7AL,556
B21 9RY,1033
B23 6DJ,599


In [32]:
total_items_max.reset_index(inplace=True)
total_items_max.head()

,post_code,items
0,B11 4BW,706
1,B12 9LP,425
2,B18 7AL,556
3,B21 9RY,1033
4,B23 6DJ,599


In [33]:
total_items = total_items_max.merge(total_items_by_postcode, on=['post_code','items'], how = 'left')

In [34]:
total_items.head()

,post_code,items,bnf_name
0,B11 4BW,706,Salbutamol_Inha 100mcg (200 D) CFF
1,B12 9LP,425,Paracet_Tab 500mg
2,B18 7AL,556,Salbutamol_Inha 100mcg (200 D) CFF
3,B21 9RY,1033,Metformin HCl_Tab 500mg
4,B23 6DJ,599,Lansoprazole_Cap 30mg (E/C Gran)


In [35]:
total_items_sorted = total_items.sort_values('bnf_name').drop_duplicates(['post_code','items'], keep='first')
total_items_sorted.head()

,post_code,items,bnf_name
73,E15 4ES,451,Amlodipine_Tab 10mg
66,DN16 2AB,1072,Amlodipine_Tab 5mg
255,WS9 8AJ,396,Amlodipine_Tab 5mg
142,NG2 7SD,605,Amlodipine_Tab 5mg
97,KT16 8HZ,401,Amlodipine_Tab 5mg


In [36]:
total_items_sorted.sort_values('post_code', inplace=True)
total_items_sorted.head()

,post_code,items,bnf_name
0,B11 4BW,706,Salbutamol_Inha 100mcg (200 D) CFF
1,B12 9LP,425,Paracet_Tab 500mg
2,B18 7AL,556,Salbutamol_Inha 100mcg (200 D) CFF
3,B21 9RY,1033,Metformin HCl_Tab 500mg
4,B23 6DJ,599,Lansoprazole_Cap 30mg (E/C Gran)


In [37]:
total_items_sorted.shape

(259, 3)

In [38]:
s = total_items_by_postcode.groupby('post_code')[['items']].sum().reset_index()
s.head()

,post_code,items
0,B11 4BW,22731
1,B12 9LP,17073
2,B18 7AL,20508
3,B21 9RY,31027
4,B23 6DJ,28011


In [39]:
s.shape

(259, 2)

In [40]:
final_df = s.merge(total_items_sorted, on='post_code')
final_df.head()

,post_code,items_x,items_y,bnf_name
0,B11 4BW,22731,706,Salbutamol_Inha 100mcg (200 D) CFF
1,B12 9LP,17073,425,Paracet_Tab 500mg
2,B18 7AL,20508,556,Salbutamol_Inha 100mcg (200 D) CFF
3,B21 9RY,31027,1033,Metformin HCl_Tab 500mg
4,B23 6DJ,28011,599,Lansoprazole_Cap 30mg (E/C Gran)


In [41]:
final_df['amt%'] = final_df['items_y'].astype(float)/final_df['items_x'].astype(float)
final_df['amt%'].head()

0    0.031059
1    0.024893
2    0.027111
3    0.033294
4    0.021384
Name: amt%, dtype: float64

In [42]:
final_df.head()

,post_code,items_x,items_y,bnf_name,amt%
0,B11 4BW,22731,706,Salbutamol_Inha 100mcg (200 D) CFF,0.031059
1,B12 9LP,17073,425,Paracet_Tab 500mg,0.024893
2,B18 7AL,20508,556,Salbutamol_Inha 100mcg (200 D) CFF,0.027111
3,B21 9RY,31027,1033,Metformin HCl_Tab 500mg,0.033294
4,B23 6DJ,28011,599,Lansoprazole_Cap 30mg (E/C Gran),0.021384


In [43]:
result = final_df[['post_code','bnf_name','amt%']]
result.head()

,post_code,bnf_name,amt%
0,B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF,0.031059
1,B12 9LP,Paracet_Tab 500mg,0.024893
2,B18 7AL,Salbutamol_Inha 100mcg (200 D) CFF,0.027111
3,B21 9RY,Metformin HCl_Tab 500mg,0.033294
4,B23 6DJ,Lansoprazole_Cap 30mg (E/C Gran),0.021384


In [44]:
result = result.head(100)
values = result.get_values().tolist()

In [45]:
final=[]

for item in values:
    final.append(tuple(item))

In [46]:
def items_by_region():
    return final

In [47]:
print(items_by_region)

<function items_by_region at 0x7ff7fb97d950>


In [48]:
#def items_by_region():
#    [("B11 4BW", "Salbutamol_Inha 100mcg (200 D) CFF", 0.0310589063)] * 100
#    [('AL1 3HD', u'Amoxicillin_Cap 500mg', 0.1026344676180022)] * 100

In [49]:
grader.score.dw__items_by_region(items_by_region)

Your score:  1.0


In [50]:
#items_by_region = [("B11 4BW", "Salbutamol_Inha 100mcg (200 D) CFF", 0.0310589063)] * 100

In [51]:
#grader.score.dw__items_by_region(items_by_region)

## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [52]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

In [53]:
opioids_join = '|'.join(opioids)
#check = '|'.join(opioids)
opioids_join

'morphine|oxycodone|methadone|fentanyl|pethidine|buprenorphine|propoxyphene|codeine'

In [54]:
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


In [55]:
opioid_codes = chem.loc[chem['NAME'].str.contains(opioids_join, case=False)]['CHEM SUB'].tolist()
len(opioid_codes)

35

In [56]:
chem_new = chem
chem_new.columns = ['bnf_code','chem_name']
chem_new.head()

,bnf_code,chem_name
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


In [57]:
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [58]:
scripts.shape

(973193, 7)

In [59]:
scrpt = scripts
scrpt.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [60]:
scrpt['opioid_prescription'] = scrpt['bnf_code'].isin(opioid_codes)
# scrpt['opioid'] = scrpt[ 'bnf_code' ].apply(lambda x: 1 if x in chem_sub else 0) #insert
# std_devn = scrpt.opioid.std () # insert
# overall_rate = scrpt.opioid.mean() # insert

In [61]:
scrpt.shape

(973193, 8)

In [62]:
scrpt.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,opioid_prescription
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,False
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,False
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,False
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,False
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,False


In [63]:
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [64]:
pract = practices[['code', 'name']]
pract.columns = ['practice', 'name']
pract.head()

,practice,name
0,A81001,THE DENSHAM SURGERY
1,A81002,QUEENS PARK MEDICAL CENTRE
2,A81003,VICTORIA MEDICAL PRACTICE
3,A81004,WOODLANDS ROAD SURGERY
4,A81005,SPRINGWOOD SURGERY


In [65]:
len(scrpt.loc[scrpt['opioid_prescription']])

34843

In [66]:
#opioids_per_practice = scrpt.groupby('practice')['opioid_prescription'].mean().rename('frac')
#opioids_per_practice.head()

In [67]:
#opioids_per_practice_std = scrpt.groupby('practice')['opioid_prescription'].std().rename('frac_std')
#opioids_per_practice_std.head()

In [68]:
#overall_rate = scrpt['opioid_prescription'].mean()
#overall_rate

In [69]:
#overall_rate_std = scrpt['opioid_prescription'].std()
#overall_rate_std

In [70]:
#relative_opioids_per_practice = (opioids_per_practice - overall_rate).rename('relative')
#relative_opioids_per_practice.head()

In [71]:
#opioid = scrpt.groupby('practice')['opioid_prescription'].sum().rename('opioid')
#opioid.head()

In [72]:
#total = scrpt.groupby('practice')['bnf_code'].count().rename('total')
#total.head()

In [73]:
#standard_error_per_practice = (overall_rate_std/(total**0.5)).rename('std_err')
#standard_error_per_practice.head()

In [74]:
#opioid_scores = (relative_opioids_per_practice/standard_error_per_practice).rename('opioid_scores')
#opioid_scores.head()

In [75]:
#merged = pd.concat([opioid, total, opioids_per_practice, relative_opioids_per_practice, standard_error_per_practice,opioid_scores], axis=1)

In [76]:
#merged = merged.reset_index()

In [77]:
#merged.head()

In [78]:
#type(merged)

In [79]:
#merged.shape

In [80]:
#pract.reset_index().head()

In [81]:
#type(pract)

In [82]:
#final_df = merged.merge(pract, on='practice', how='left')

In [83]:
#final_df.head()

In [84]:
#final_df.sort_values('opioid_scores', ascending = False , inplace=True)

In [85]:
#final_df.head()

In [86]:
#final = final_df.drop_duplicates('name')
#final.head()

In [87]:
#result = final[['name','opioid_scores','total']]
#result.head()

In [88]:
#result = result.head(100)
#values = result.get_values().tolist()

In [89]:
#answer=[]

#for item in values:
#    answer.append(tuple(item))

In [90]:
#def script_anomalies():
#    return answer

In [91]:
#print(script_anomalies)

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [92]:
# scripts_chem = pd.merge(scripts, chem, left_on='bnf_code', right_on='chem_name')
# scripts_chem.head()
scrpt = scrpt.merge (practices, left_on = 'practice', right_on = 'code') #insert
scrpt['cnt'] = 0 #insert


Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [93]:
opioids_per_practice = ...

In [94]:
opioids_per_practice = scrpt.groupby('practice')['opioid_prescription'].mean().rename('frac')
opioids_per_practice.head()
# opioids_per_practice = scrpt.groupby ( [ 'practice', 'name' ], as_index = False ).agg ( { 'opioid': 'mean', 'cnt': 'count' } ) #insert
# opioids_per_practice.drop_duplicates() # insert


# opioids_per_practice['opioid'] = opioids_per_practice ['opioid'] - overall_rate # insert

# opioids_per_practice['std_err'] = std_devn / opioids_per_practice['cnt'] ** 0.5 # insert
# opioids_per_practice['z_score'] = opioids_per_practice['opioid'] / opioids_per_practice['std_err'] # insert


practice
A81005    0.033179
A81007    0.043329
A81011    0.046556
A81012    0.042793
A81017    0.038140
Name: frac, dtype: float64

In [95]:
#
#result = opioids_per_practice[['name', 'z_score', 'cnt']]

In [96]:
#
# result.sort_values(by = 'z_score', ascending = False, inplace = True)
# anomalies = [(k[1], k[2], k[3]) for k in result.itertuples()][:100]

In [97]:
opioids_per_practice_std = scrpt.groupby('practice')['opioid_prescription'].std().rename('frac_std')
opioids_per_practice_std.head()

practice
A81005    0.179162
A81007    0.203666
A81011    0.210753
A81012    0.202466
A81017    0.191578
Name: frac_std, dtype: float64

In [98]:
overall_rate = scrpt['opioid_prescription'].mean()
overall_rate

0.035797347828884114

In [99]:
overall_rate_std = scrpt['opioid_prescription'].std()
overall_rate_std

0.18578462836316456

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [100]:
relative_opioids_per_practice = ...

In [101]:
relative_opioids_per_practice = (opioids_per_practice - overall_rate).rename('relative')
relative_opioids_per_practice.head()

practice
A81005   -0.002619
A81007    0.007531
A81011    0.010759
A81012    0.006995
A81017    0.002342
Name: relative, dtype: float64

In [102]:
opioid = scrpt.groupby('practice')['opioid_prescription'].sum().rename('opioid')
opioid.head()

practice
A81005    50.0
A81007    63.0
A81011    73.0
A81012    57.0
A81017    82.0
Name: opioid, dtype: float64

In [103]:
total = scrpt.groupby('practice')['bnf_code'].count().rename('total')
total.head()

practice
A81005    1507
A81007    1454
A81011    1568
A81012    1332
A81017    2150
Name: total, dtype: int64

In [104]:
#
#total_prescriptions = merged_chem.groupby('practice').items.count()
# total_prescriptions = scrpt.groupby('practice').size()
# #flagged_prescriptions = merged_chem[merged_chem['flag']].groupby('practice').flag.count()  
# scrpt['flag'] = scripts_chem['flag']
# scrpt['flag'].fillna(False, inplace=True)

In [105]:
#
# opioids_per_practice = flagged_prescriptions / total_prescriptions
# relative_opioids_per_practice = opioids_per_practice - (flagged_prescriptions.sum() / total_prescriptions.sum())

In [106]:
#
# standard_error_per_practice = scripts.flag.std() / (total_prescriptions ** .5)

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [107]:
standard_error_per_practice = ...
opioid_scores = ...

In [108]:
#
#total_prescriptions = scripts.groupby('practice').size()
# total = scripts.groupby('practice').size()

In [109]:
#
# scripts['flag'] = merged_chem['flag']
# scripts['flag'].fillna(False, inplace=True)
#scrpt['flag'] = merged_chem['flag']
# scrpt['flag'] = scripts_chem['flag']
# scrpt['flag'].fillna(False, inplace=True)

In [110]:
#
#standard_error_per_practice = scripts.flag.std() / (total_prescriptions ** .5)
# standard_error_per_practice = scrpt.flag.std() / (total ** .5)

In [111]:
standard_error_per_practice = (overall_rate_std/(total**0.5)).rename('std_err')
standard_error_per_practice.head()

practice
A81005    0.004786
A81007    0.004872
A81011    0.004692
A81012    0.005090
A81017    0.004007
Name: std_err, dtype: float64

In [112]:
opioid_scores = (relative_opioids_per_practice/standard_error_per_practice).rename('opioid_scores')
# opioid_scores = (relative_opioids_per_practice/standard_error_per_practice).rename('z_scores') #insert

opioid_scores.head()

practice
A81005   -0.547214
A81007    1.545782
A81011    2.293116
A81012    1.374224
A81017    0.584563
Name: opioid_scores, dtype: float64

In [113]:
merged = pd.concat([opioid, total, opioids_per_practice, relative_opioids_per_practice, standard_error_per_practice,opioid_scores], axis=1)

In [114]:
merged = merged.reset_index()

In [115]:
merged.head()

,practice,opioid,total,frac,relative,std_err,opioid_scores
0,A81005,50.0,1507,0.033179,-0.002619,0.004786,-0.547214
1,A81007,63.0,1454,0.043329,0.007531,0.004872,1.545782
2,A81011,73.0,1568,0.046556,0.010759,0.004692,2.293116
3,A81012,57.0,1332,0.042793,0.006995,0.005090,1.374224
4,A81017,82.0,2150,0.038140,0.002342,0.004007,0.584563


In [116]:
type(merged)

pandas.core.frame.DataFrame

In [117]:
merged.shape

(856, 7)

In [118]:
pract.reset_index().head()

,index,practice,name
0,0,A81001,THE DENSHAM SURGERY
1,1,A81002,QUEENS PARK MEDICAL CENTRE
2,2,A81003,VICTORIA MEDICAL PRACTICE
3,3,A81004,WOODLANDS ROAD SURGERY
4,4,A81005,SPRINGWOOD SURGERY


In [119]:
type(pract)

pandas.core.frame.DataFrame

In [120]:
final_df = merged.merge(pract, on='practice', how='left')

In [121]:
final_df.head()

,practice,opioid,total,frac,relative,std_err,opioid_scores,name
0,A81005,50.0,1507,0.033179,-0.002619,0.004786,-0.547214,SPRINGWOOD SURGERY
1,A81007,63.0,1454,0.043329,0.007531,0.004872,1.545782,BANKHOUSE SURGERY
2,A81011,73.0,1568,0.046556,0.010759,0.004692,2.293116,CHADWICK PRACTICE
3,A81012,57.0,1332,0.042793,0.006995,0.005090,1.374224,WESTBOURNE MEDICAL CENTRE
4,A81017,82.0,2150,0.038140,0.002342,0.004007,0.584563,WOODBRIDGE PRACTICE


In [122]:
final_df.sort_values('opioid_scores', ascending = False , inplace=True)
# final_df.sort_values('z_scores', ascending = False , inplace=True)

In [123]:
final_df.head()

,practice,opioid,total,frac,relative,std_err,opioid_scores,name
829,Y01852,6.0,7,0.857143,0.821346,0.070220,11.696748,NATIONAL ENHANCED SERVICE
874,Y03006,2.0,2,1.000000,0.964203,0.131370,7.339619,OUTREACH SERVICE NH / RH
895,Y03668,11.0,60,0.183333,0.147536,0.023985,6.151256,BRISDOC HEALTHCARE SERVICES OOH
296,G81703,7.0,36,0.194444,0.158647,0.030964,5.123581,H&R P C SPECIAL SCHEME
948,Y04997,28.0,321,0.087227,0.051430,0.010369,4.959750,HMR BARDOC OOH


In [124]:
final = final_df.drop_duplicates('name')
final.head()

,practice,opioid,total,frac,relative,std_err,opioid_scores,name
829,Y01852,6.0,7,0.857143,0.821346,0.070220,11.696748,NATIONAL ENHANCED SERVICE
874,Y03006,2.0,2,1.000000,0.964203,0.131370,7.339619,OUTREACH SERVICE NH / RH
895,Y03668,11.0,60,0.183333,0.147536,0.023985,6.151256,BRISDOC HEALTHCARE SERVICES OOH
296,G81703,7.0,36,0.194444,0.158647,0.030964,5.123581,H&R P C SPECIAL SCHEME
948,Y04997,28.0,321,0.087227,0.051430,0.010369,4.959750,HMR BARDOC OOH


In [125]:
result = final[['name','opioid_scores','total']]
# result = final[['name','z_scores','total']]

result.head()

,name,opioid_scores,total
829,NATIONAL ENHANCED SERVICE,11.696748,7
874,OUTREACH SERVICE NH / RH,7.339619,2
895,BRISDOC HEALTHCARE SERVICES OOH,6.151256,60
296,H&R P C SPECIAL SCHEME,5.123581,36
948,HMR BARDOC OOH,4.959750,321


In [126]:
result = result.head(100)
values = result.get_values().tolist()

In [127]:
answer=[]

for item in values:
    answer.append(tuple(item))

In [128]:
def script_anomalies():
    return answer

In [129]:
print(script_anomalies)

<function script_anomalies at 0x7ff7fb9d99d8>


In [130]:
#
# result = opioids_per_practice[['name', 'z_score', 'cnt']]

In [131]:
#
result.sort_values(by = 'opioid_scores', ascending = False, inplace = True)
anomalies = [(k[1], k[2], k[3]) for k in result.itertuples()][:100]
# anomalies = [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

In [132]:
# unique_practices = ...
# anomalies = [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

In [133]:
grader.score.dw__script_anomalies(anomalies)

Your score:  0.7000000000000004


The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [134]:
unique_practices = ...
anomalies = ...

In [135]:
def script_anomalies():
    return [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

In [136]:
grader.score('dw__script_anomalies', script_anomalies)

Error!
You have been rate limited for exceeding the limit of 1 per 1 second.
Please slow down your submission rate.


In [137]:
#unique_practices = ...
#anomalies = [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

In [138]:
#grader.score.dw__script_anomalies(anomalies)

## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [139]:
drugs_16=scripts_16[['bnf_name', 'items']]
drugs_16=drugs_16.groupby('bnf_name').count().reset_index().drop_duplicates()
drugs_16.columns=[['bnf_name', 'count_16']]
drugs_16.head()

,bnf_name,count_16
0,365 Film 15cm x 20cm VP Adh Film Dress,1
1,365 Film 4cm x 5cm VP Adh Film Dress,1
2,365 Non Adherent 10cm x 10cm Pfa Plas Fa,3
3,365 Non Woven Island 10cm x 10cm Adh Dre,1
4,365 Non Woven Island 10cm x 15cm Adh Dre,1


In [140]:
drugs_17=scripts[['bnf_name', 'items']]
drugs_17=drugs_17.groupby('bnf_name').count().reset_index().drop_duplicates()
drugs_17.columns=[['bnf_name', 'count_17']]
drugs_17.head()

,bnf_name,count_17
0,365 Film 10cm x 12cm VP Adh Film Dress,1
1,365 Non Adherent 10cm x 10cm Pfa Plas Fa,3
2,365 Non Adherent 10cm x 20cm Pfa Plas Fa,1
3,365 Non Woven Island 8cm x 10cm Adh Dres,1
4,365 Transpt Island 5cm x 7.2cm VP Adh Fi,2


In [141]:
total_16=drugs_16['count_16'].sum()
total_17=drugs_17['count_17'].sum()

In [142]:
drugs_16['rate_16']=drugs_16['count_16']/total_16.astype(float)
drugs_17['rate_17']=drugs_17['count_17']/total_17.astype(float)

In [143]:
drugs=drugs_16.merge(drugs_17, on='bnf_name', how='inner').drop_duplicates('bnf_name')
drugs.head()

,bnf_name,count_16,rate_16,count_17,rate_17
0,365 Non Adherent 10cm x 10cm Pfa Plas Fa,3,0.000003,3,0.000003
1,365 Transpt Island 8.5cm x 15.5cm VP Adh,1,0.000001,1,0.000001
2,3m Health Care_Cavilon Durable Barrier C,825,0.000848,816,0.000838
3,3m Health Care_Cavilon No Sting 1ml Barr,231,0.000237,223,0.000229
4,3m Health Care_Cavilon No Sting 3ml Barr,105,0.000108,94,0.000097


In [144]:
drugs['growth']=(drugs['rate_17']-drugs['rate_16'])/drugs['rate_16'].astype(float)
drugs.head()

,bnf_name,count_16,rate_16,count_17,rate_17,growth
0,365 Non Adherent 10cm x 10cm Pfa Plas Fa,3,0.000003,3,0.000003,0.000000
1,365 Transpt Island 8.5cm x 15.5cm VP Adh,1,0.000001,1,0.000001,0.000000
2,3m Health Care_Cavilon Durable Barrier C,825,0.000848,816,0.000838,-0.010909
3,3m Health Care_Cavilon No Sting 1ml Barr,231,0.000237,223,0.000229,-0.034632
4,3m Health Care_Cavilon No Sting 3ml Barr,105,0.000108,94,0.000097,-0.104762


In [145]:
total_growth=(total_17-total_16)/total_16.astype(float)
total_growth

0.0

In [146]:
drugs['rel_growth']=drugs['growth']+total_growth
drugs=drugs[(drugs['count_16']>50)]
drugs = drugs[['bnf_name', 'rel_growth', 'count_16']]
drugs.head()

,bnf_name,rel_growth,count_16
2,3m Health Care_Cavilon Durable Barrier C,-0.010909,825
3,3m Health Care_Cavilon No Sting 1ml Barr,-0.034632,231
4,3m Health Care_Cavilon No Sting 3ml Barr,-0.104762,105
5,3m Health Care_Cavilon No Sting Barrier,-0.092511,454
8,A.S Saliva Orthana Spy 50ml (App),0.440476,84


In [147]:
drugs.sort_values('rel_growth', ascending=False, inplace=True)
drugs.head()

,bnf_name,rel_growth,count_16
1467,Butec_Transdermal Patch 5mcg/hr,3.467742,62
1465,Butec_Transdermal Patch 10mcg/hr,3.000000,69
4415,Fostair NEXThaler_Inh 200mcg/6mcg (120D),1.430233,86
8498,Pneumococcal_Vac 0.5ml Vl (23 Valent),1.269430,193
9993,Spiolto Respimat_Inha2.5/2.5mcg(60D)+Dev,1.269231,52


In [148]:
final = pd.concat([drugs.iloc[0:50], drugs.iloc[len(drugs)-50: len(drugs)]], axis=0)
final.head()

,bnf_name,rel_growth,count_16
1467,Butec_Transdermal Patch 5mcg/hr,3.467742,62
1465,Butec_Transdermal Patch 10mcg/hr,3.000000,69
4415,Fostair NEXThaler_Inh 200mcg/6mcg (120D),1.430233,86
8498,Pneumococcal_Vac 0.5ml Vl (23 Valent),1.269430,193
9993,Spiolto Respimat_Inha2.5/2.5mcg(60D)+Dev,1.269231,52


In [149]:
arr = final.values
arr[:5]

array([['Butec_Transdermal Patch 5mcg/hr', 3.467741935483871, 62],
       ['Butec_Transdermal Patch 10mcg/hr', 3.0, 69],
       ['Fostair NEXThaler_Inh 200mcg/6mcg (120D)', 1.4302325581395348,
        86],
       ['Pneumococcal_Vac 0.5ml Vl (23 Valent)', 1.2694300518134713, 193],
       ['Spiolto Respimat_Inha2.5/2.5mcg(60D)+Dev', 1.2692307692307692,
        52]], dtype=object)

In [150]:
lst=[]

for item in arr:
    lst.append(tuple(item))

In [151]:
def script_growth():
    return lst

In [152]:
grader.score('dw__script_growth', script_growth)

Your score:  0.9900000000000007


In [ ]:
#scripts16 = ...

In [ ]:
#script_growth = [("Butec_Transdermal Patch 5mcg\/hr", 3.4677419355, 62.0)] * 100

In [ ]:
#grader.score.dw__script_growth(script_growth)

## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [ ]:
p = ...
rates = ...
rare_codes = ...
scripts['rare'] = ...

In [153]:
# Calculating probability for each bnf_code
p = 1 / float(scripts['bnf_code'].nunique())
# p < 0.1*P #. you will get 844 rows


In [154]:
# Calculating rare prescription i.e ratio of count of bnf_code per prescription and total count

rates = scripts.groupby('bnf_code')['bnf_code'].count().rename('count_prescription').reset_index()
rates.head()

,bnf_code,count_prescription
0,0101010C0,34
1,0101010F0,2
2,0101010G0,375
3,0101010I0,34
4,0101010J0,41


In [155]:
total_count = scripts['bnf_code'].count().astype(float)
total_count

973193.0

In [156]:
rare = rates['count_prescription'] / total_count
rare.head()

0    0.000035
1    0.000002
2    0.000385
3    0.000035
4    0.000042
Name: count_prescription, dtype: float64

In [157]:
#Filtering the records on the basis of  rate < 0.1p

rare_codes =  rates[rare < 0.1*p]['bnf_code'].tolist()
# rare_codes[:5]
rare_codes

['0101010C0',
 '0101010F0',
 '0101010I0',
 '0101010J0',
 '0101010N0',
 '0101010P0',
 '0101010Q0',
 '0101012B0',
 '010102100',
 '0101021C0',
 '0102000AC',
 '0102000D0',
 '010300000',
 '0103020P0',
 '0103030W0',
 '0103040M0',
 '0104020H0',
 '0104020N0',
 '0104020P0',
 '0104030A0',
 '0105020C0',
 '0105020G0',
 '010602000',
 '0106020B0',
 '0106020J0',
 '0106030A0',
 '0106030P0',
 '0106040J0',
 '0106040L0',
 '0106040X0',
 '0106050B0',
 '0106050M0',
 '0106060B0',
 '0106070C0',
 '010701000',
 '0108010S0',
 '0109010R0',
 '0202010D0',
 '0202010J0',
 '0202010L0',
 '0202010Y0',
 '0202040G0',
 '0202040H0',
 '0202040T0',
 '0202040U0',
 '0202040V0',
 '0203020G0',
 '0203020P0',
 '0203020U0',
 '020400030',
 '0204000N0',
 '0204000P0',
 '0205010AB',
 '0205010Y0',
 '0205040I0',
 '0205040M0',
 '0205051AA',
 '0205051AB',
 '0205051AC',
 '0205051E0',
 '0205051G0',
 '0205051P0',
 '0205051S0',
 '0205052AC',
 '0205052AD',
 '0206020B0',
 '0206020I0',
 '0206020M0',
 '0206040AF',
 '0206040AI',
 '0207030A0',
 '0208

In [158]:
#Creating 'rare' column in scripts
scrpt = scripts
scrpt['rare'] = scrpt['bnf_code'].isin(rare_codes)

In [159]:
scrpt.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,opioid_prescription,rare
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,False,False
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,False,False
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,False,False
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,False,False
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,False,False


In [160]:
scrpt.shape

(973193, 9)

In [ ]:
# Calculate total sum of act_cost for all treatment per prescription
#treatment_sum = scrpt.groupby('practice')['act_cost'].sum()
#treatment_sum.head()

In [ ]:
# Calculate cost for rare treatment per prescription
#rare_treatment_sum = scrpt[scrpt['rare']].groupby('practice')['act_cost'].sum()
#rare_treatment_sum.head()

In [ ]:
# Calculate proportion of costs that originate from rare treatment i.e ratio of rare_treatment_sum and treatment_sum

#rare_cost_prop = (rare_treatment_sum / treatment_sum)
#rare_cost_prop.fillna(0, inplace=True)

#mean_val = scrpt[scrpt['rare']]['act_cost'].sum() / scrpt['act_cost'].sum()

#relative_rare_cost_prop = rare_cost_prop - mean_val 
#standard_errors = relative_rare_cost_prop.std()

#mean_val, standard_errors

In [ ]:
#rare_scores = (relative_rare_cost_prop / standard_errors).rename('z-score').reset_index()
#rare_scores.columns =['code','z-score']
#rare_scores.head()

In [ ]:
# Sorting rare_score in descendig order of z-score
#rare_scores.sort_values('z-score', ascending=False, inplace=True)
#rare_scores.columns =['code','z-score']
#rare_scores.head()

In [ ]:
#practices.head()

In [ ]:
#pract = practices[['code','name']].drop_duplicates('code')
#pract.head()

In [ ]:
#pract.shape

In [ ]:
#rare_scores.shape

In [ ]:
#Joining rare_scores with practices to get (practice, name, z-score)
#joined_data = pd.merge(rare_scores, pract, on='code', how ='inner')#.drop_duplicates(['code','name'])
#joined_data.head()

In [ ]:
#joined_data.shape

In [ ]:
#joined_data.sort_values('z-score', ascending=False, inplace=True)

In [ ]:
#joined_data.reset_index(inplace=True)

In [ ]:
#required_cols = ['practice_code', 'practice_name', 'z-score']
#required_cols = ['code', 'name', 'z-score']
#final_df = joined_data[required_cols]

In [ ]:
#final_df.head()

In [ ]:
#final_df.shape

In [ ]:
#final_df = final_df.head(100)
#arr = final_df.values

In [ ]:
#lst=[]

#for item in arr:
#    lst.append(tuple(item))

In [ ]:
#def rare_scripts():
#    return lst

In [ ]:
#def rare_scripts():
    #return [("Y03106", "CRI GPWSI", 14.516)] * 100
    #return [tuple(i) for i in final_df.values][:100]

In [ ]:
#rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100
#rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.262687)] * 100

In [ ]:
#grader.score.dw__rare_scripts(rare_scripts)
#grader.score('dw__rare_scripts', rare_scripts)

Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [ ]:
rare_cost_prop = ...

In [161]:
# Calculate total sum of act_cost for all treatment per prescription
treatment_sum = scrpt.groupby('practice')['act_cost'].sum()
# treatment_sum.head()

In [162]:
# Calculate cost for rare treatment per prescription
rare_treatment_sum = scrpt[scrpt['rare']].groupby('practice')['act_cost'].sum()
# rare_treatment_sum.head()

In [163]:
# Calculate proportion of costs that originate from rare treatment i.e ratio of rare_treatment_sum and treatment_sum

rare_cost_prop = (rare_treatment_sum / treatment_sum)
rare_cost_prop.fillna(0, inplace=True)

mean_val = scrpt[scrpt['rare']]['act_cost'].sum() / scrpt['act_cost'].sum()

#relative_rare_cost_prop = rare_cost_prop - mean_val 
#standard_errors = relative_rare_cost_prop.std()

#mean_val, standard_errors

In [ ]:
#rare_scores = (relative_rare_cost_prop / standard_errors).rename('z-score').reset_index()
#rare_scores.columns =['code','z-score']
#rare_scores.head()

In [ ]:
# Sorting rare_score in descendig order of z-score
#rare_scores.sort_values('z-score', ascending=False, inplace=True)
#rare_scores.columns =['code','z-score']
#rare_scores.head()

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [ ]:
relative_rare_cost_prop = ...

In [164]:
# Calculate proportion of costs that originate from rare treatment i.e ratio of rare_treatment_sum and treatment_sum

#rare_cost_prop = (rare_treatment_sum / treatment_sum)
#rare_cost_prop.fillna(0, inplace=True)

#mean_val = scrpt[scrpt['rare']]['act_cost'].sum() / scrpt['act_cost'].sum()

relative_rare_cost_prop = rare_cost_prop - mean_val
relative_rare_cost_prop.head()
#standard_errors = relative_rare_cost_prop.std()

#mean_val, standard_errors

practice
A81005   -0.003946
A81007   -0.007582
A81011   -0.010847
A81012   -0.002216
A81017   -0.008604
Name: act_cost, dtype: float64

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [ ]:
standard_errors = ...

In [165]:
# Calculate proportion of costs that originate from rare treatment i.e ratio of rare_treatment_sum and treatment_sum

#rare_cost_prop = (rare_treatment_sum / treatment_sum)
#rare_cost_prop.fillna(0, inplace=True)

#mean_val = scrpt[scrpt['rare']]['act_cost'].sum() / scrpt['act_cost'].sum()

#relative_rare_cost_prop = rare_cost_prop - mean_val 
standard_errors = relative_rare_cost_prop.std()

mean_val, standard_errors

(0.015962901360950386, 0.06050888706745139)

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [ ]:
rare_scores = ...

In [166]:
rare_scores = (relative_rare_cost_prop / standard_errors).rename('z-score').reset_index()
rare_scores.columns =['code','z-score']
rare_scores.head()

,code,z-score
0,A81005,-0.065216
1,A81007,-0.125308
2,A81011,-0.179263
3,A81012,-0.036615
4,A81017,-0.142190


In [167]:
# Sorting rare_score in descendig order of z-score
rare_scores.sort_values('z-score', ascending=False, inplace=True)
rare_scores.columns =['code','z-score']
rare_scores.head()

,code,z-score
765,Y03472,16.262687
831,Y05320,15.128648
793,Y04404,7.542139
766,Y03484,7.287222
794,Y04424,6.838614


In [168]:
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [189]:
pract = practices[['code','name']].drop_duplicates('code')
# pract = practices[['code','name']].drop_duplicates('name',) #change duplicate code to name
pract.head()

,code,name
0,A81001,THE DENSHAM SURGERY
1,A81002,QUEENS PARK MEDICAL CENTRE
2,A81003,VICTORIA MEDICAL PRACTICE
3,A81004,WOODLANDS ROAD SURGERY
4,A81005,SPRINGWOOD SURGERY


In [190]:
pract.shape

(10843, 2)

In [191]:
rare_scores.shape

(856, 2)

In [192]:
#Joining rare_scores with practices to get (practice, name, z-score)
joined_data = pd.merge(rare_scores, pract, on='code', how ='inner')#.drop_duplicates(['code','name'])

# joined_data = pd.merge(rare_scores, pract, on='code', how ='inner')#.drop_duplicates(['code','name'])

# merged_df = pd.merge(rare_codes,scripts2,left_on='bnf_code',right_on='bnf_code') #insert

joined_data.head()

,code,z-score,name
0,Y03472,16.262687,CONSULTANT DIABETES TEAM
1,Y05320,15.128648,DMC COMMUNITY DERMATOLOGY RBWF
2,Y04404,7.542139,OUTPATIENTS JUBILEE HEALTH CENTRE
3,Y03484,7.287222,DMC COMMUNITY DERMATOLOGY CLINIC
4,Y04424,6.838614,DMC HEALTHCARE


In [193]:
joined_data.shape

(856, 3)

In [194]:
joined_data.sort_values('z-score', ascending=False, inplace=True)
# joined_data.sort_values('code', ascending=False, inplace=True)

# final_df.sort_values('opioid_scores', ascending = False , inplace=True)

In [195]:
joined_data.reset_index(inplace=True)

In [196]:
#required_cols = ['practice_code', 'practice_name', 'z-score']
required_cols = ['code', 'name', 'z-score']
final_df = joined_data[required_cols]

In [177]:
final_df.head()

,code,name,z-score
0,Y03472,CONSULTANT DIABETES TEAM,16.262687
1,Y03472,CPFT - CONSULTANT DIABETES TEAM,16.262687
2,Y05320,DMC COMMUNITY DERMATOLOGY RBWF,15.128648
3,Y04404,OUTPATIENTS JUBILEE HEALTH CENTRE,7.542139
4,Y03484,DMC COMMUNITY DERMATOLOGY CLINIC,7.287222


In [178]:
final_df.shape

(859, 3)

In [ ]:
# final_df = final_df.drop_duplicates('name') #insert

In [ ]:
# final_df.shape #insert

In [197]:
final_df = final_df.head(100)
arr = final_df.values

In [199]:
lst=[]

for item in arr:
    lst.append(tuple(item))

In [202]:
def rare_scripts():
    return lst

In [186]:
# rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100

In [203]:
#grader.score.dw__rare_scripts(rare_scripts)
grader.score('dw__rare_scripts', rare_scripts)

Your score:  0.9300000000000006


In [ ]:
#def rare_scripts():
#    return [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100
    #return [("Y03106", "CRI GPWSI", 14.516)] * 100
#    rare_scores.sort_values('act_cost', ascending=False, inplace=True)

In [ ]:
#grader.score('dw__rare_scripts', rare_scripts)

In [ ]:
#rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100

In [ ]:
#grader.score.dw__rare_scripts(rare_scripts)

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*